# Mapping Previous Test Results
### Calum Macdonald
## Create mapping indices to reorder Tia's results in h5 test set ordering

In [7]:
import sys
import os
import time
import math
import random
import pdb
import h5py
import numpy as np
from progressbar import *
from tqdm.notebook import tnrange

In [8]:
# Dictionary mapping the ordinal labels to particle types 
LABEL_DICT = {0:"gamma", 1:"e", 2:"mu"}

# Fix the colour scheme for each particle type
COLOR_DICT = {"gamma":"red", "e":"blue", "mu":"green"}

### Load Tia's data

In [9]:
fprs = []
tprs = []
thresholds = []

run_id = "/resnet_results"

dump_dir = "/data/WatChMaL/data"
dump_file = "/test_validation_iteration_dump.npz"

softmax_index_dict = {value:key for key, value in LABEL_DICT.items()}
    
test_dump_path = dump_dir + run_id + dump_file
test_dump_np = np.load(test_dump_path, allow_pickle=True)

res_predictedlabels = np.concatenate(list([batch_array for batch_array in test_dump_np['predicted_labels']]))
res_softmaxes  = np.concatenate(list([batch_array for batch_array in test_dump_np['softmax']]))
res_labels   = np.concatenate(list([batch_array for batch_array in test_dump_np['labels']]))
res_energies = np.concatenate(list([batch_array for batch_array in test_dump_np['energies']]))
res_rootfiles = np.concatenate(list([batch_array for batch_array in test_dump_np['rootfiles']]))
res_eventids = np.concatenate(list([batch_array for batch_array in test_dump_np['eventids']]))
#res_positions = test_dump_np['positions'].reshape(-1)
res_angles = np.concatenate(list([batch_array for batch_array in test_dump_np['angles']]))

### Load h5 data

In [10]:
# Import test events from h5 file
filtered_index = "/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz"
filtered_indices = np.load(filtered_index, allow_pickle=True)
test_filtered_indices = filtered_indices['test_idxs']
print(test_filtered_indices.shape)

original_data_path = "/data/WatChMaL/data/IWCDmPMT_4pi_fulltank_9M.h5"
f = h5py.File(original_data_path, "r")

hdf5_event_data = (f["event_data"])
original_eventdata = np.memmap(original_data_path, mode="r", shape=hdf5_event_data.shape,
                                    offset=hdf5_event_data.id.get_offset(), dtype=hdf5_event_data.dtype)

original_eventids = np.array(f['event_ids'])
original_rootfiles = np.array(f['root_files'])
original_energies = np.array(f['energies'])
original_positions = np.array(f['positions'])
original_angles = np.array(f['angles'])
original_labels = np.array(f['labels'])

#filtered_eventdata = original_eventdata[test_filtered_indices]
filtered_eventids = original_eventids[test_filtered_indices]
filtered_rootfiles = original_rootfiles[test_filtered_indices]
filtered_energies = original_energies[test_filtered_indices]
filtered_positions = original_positions[test_filtered_indices]
filtered_angles = original_angles[test_filtered_indices]
filtered_labels = original_labels[test_filtered_indices]

(3351020,)


### Create mapping indices

In [5]:
mapping_indices = np.array([])
for i in tnrange(filtered_rootfiles.shape[0]):
    id_index = np.where(res_eventids == filtered_eventids[i])[0]
    mapping_indices = np.append(mapping_indices, id_index[np.where((res_rootfiles[id_index] == filtered_rootfiles[i]))[0]])
np.savez(os.path.join(os.getcwd(),'map_indices_previous_resnet'), mapping_indices)    

### Check Them

In [14]:
mapping_indices = np.load(os.path.join(os.getcwd(),'/Index_Storage/map_indices_previous_resnet.npz'),allow_pickle=True)['arr_0']
ordered_rn_eventids = res_eventids[mapping_indices.astype(int)]
ordered_rn_rootfiles = res_rootfiles[mapping_indices.astype(int)]

In [18]:
for i in tnrange(mapping_indices.shape[0]):
    assert filtered_eventids[i] == ordered_rn_eventids[i]
    assert filtered_rootfiles[i] == ordered_rn_rootfiles[i]
print('Success! We have mapping indices to reorder Tia\'s results.')


Success! We have mapping indices to reorder Tia's results.
